In [1]:
%pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [7]:
import requests
import pandas as pd
from bs4 import  BeautifulSoup


### Wikipedia



In [6]:
url = "https://fr.wikipedia.org/wiki/D%C3%A9mographie_de_l%27%C3%8Ele-de-France"
headers = {"User-Agent": "ENSAE-coursework", "Accept-Language": "fr"}
html = requests.get(url, headers=headers, timeout=30).text
df_wiki = pd.read_html(html, attrs={"class": "wikitable centre"}, header=1)[0]
df_wiki.columns = [c.split('[')[0].strip() for c in df_wiki.columns]

df_wiki.tail(10)

/tmp/ipykernel_21354/2190363457.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_wiki = pd.read_html(html, attrs={"class": "wikitable centre"}, header=1)[0]


,Année,Paris,Seine-et-Marne,Yvelines,Essonne,Hauts-de-Seine,Seine-Saint-Denis,Val-de-Marne,Val-d'Oise,Île-de-France
27,2014,2 220 445,1 377 846,1 421 670,1 268 228,1 597 770,1 571 028,1 365 039,1 205 539,12 027 565
28,2015,2 206 488,1 390 121,1 427 291,1 276 233,1 601 569,1 592 663,1 372 389,1 215 390,12 082 144
29,2016,2 190 327,1 397 665,1 431 808,1 287 330,1 603 268,1 606 660,1 378 151,1 221 923,12 117 132
30,2017,2 187 526,1 403 997,1 438 266,1 296 130,1 609 306,1 623 111,1 387 926,1 228 618,12 174 880
31,2018,2 175 601,1 412 516,1 441 398,1 296 641,1 619 120,1 632 677,1 396 913,1 238 581,12 213 447
32,2019,2 165 423,1 421 197,1 448 207,1 301 659,1 624 357,1 644 903,1 407 124,1 249 674,12 262 544
33,2020,2 145 906,1 428 636,1 449 723,1 306 118,1 626 213,1 655 422,1 407 972,1 251 804,12 271 794
34,2021,2 133 111,1 438 100,1 456 365,1 313 768,1 635 291,1 668 670,1 415 367,1 256 607,12 317 279
35,2022,2 113 705,1 452 399,1 470 778,1 324 546,1 647 435,1 681 725,1 419 531,1 270 845,12 380 964
36,Année,Paris,Seine-et-Marne,Yvelines,Essonne,Hauts-de-Seine,Seine-Saint-Denis,Val-de-Marne,Val-d'Oise,Île-de-France


### Table population 2023-2024 from INSEE 

In [8]:
page_url = "https://www.insee.fr/fr/statistiques/8062506"
read_html = requests.get(page_url)
soup = BeautifulSoup(read_html.text,"html.parser")

In [9]:
table = soup.find_all("table",id = "produit-tableau-figure1")
tbody = soup.find("tbody")

In [11]:
rows = []
for tr in tbody.find_all("tr"):
    cells = [td.get_text() for td in tr.find_all(["th","td"])]
    if not cells:
        continue
    territoire = cells[0]
    rows.append({"Territoire":territoire, 
                "2023":cells[1],
                "2024":cells[2]})
df_insee = pd.DataFrame(rows)
df_insee.T

,0,1,2,3,4,5,6,7,8,9,10
Territoire,Paris,Hauts-de-Seine,Seine-Saint-Denis,Val-de-Marne,Seine-et-Marne,Yvelines,Essonne,Val-d’Oise,Île-de-France,France métropolitaine,France
2023,2 087 600,1 651 400,1 701 000,1 433 900,1 464 800,1 473 700,1 331 800,1 275 700,12 419 900,66 142 900,68 373 400
2024,2 104 300,1 646 400,1 690 400,1 428 100,1 455 800,1 467 800,1 325 900,1 269 700,12 388 400,65 926 000,68 143 400


### Join these two tables by using department 

In [12]:
df_wiki.rename(columns={"Année": "Annee", "Val-d'Oise":"Val-d’Oise"}, inplace=True)
df_insee_t = df_insee.set_index("Territoire").T.reset_index()
df_insee_t.rename(columns={"index": "Annee"}, inplace=True)

df_insee_t["Annee"] = df_insee_t["Annee"].astype(int)

common_cols = ["Annee"] + [c for c in df_wiki.columns if c != "Annee" and c in df_insee_t.columns]
df_insee_t = df_insee_t[common_cols]
df_final = pd.concat([df_wiki, df_insee_t], ignore_index=True)
df_final = df_final[df_final["Annee"].apply(lambda x: str(x).isdigit())]
df_final
# df_final = df_final.sort_values("Annee").reset_index(drop=True)

,Annee,Paris,Seine-et-Marne,Yvelines,Essonne,Hauts-de-Seine,Seine-Saint-Denis,Val-de-Marne,Val-d’Oise,Île-de-France
0,1968,2 590 771,604 340,854 382,673 325,1 461 619,1 249 606,1 121 319,693 269,9 248 931
1,1975,2 299 830,755 762,1 082 296,923 061,1 438 930,1 322 127,1 215 674,840 885,9 878 565
2,1982,2 176 243,887 112,1 196 111,987 988,1 387 039,1 324 301,1 193 655,920 598,10 073 059
3,1990,2 150 147,1 077 206,1 305 076,1 083 245,1 389 416,1 378 576,1 213 446,1 047 553,10 644 665
4,1991,2 145 912,1 095 235,1 314 925,1 093 658,1 392 369,1 380 953,1 215 154,1 057 350,10 695 556
5,1992,2 139 928,1 115 222,1 326 185,1 105 974,1 394 910,1 384 908,1 218 450,1 067 699,10 753 276
6,1993,2 132 475,1 131 179,1 333 814,1 115 400,1 395 461,1 387 427,1 221 158,1 076 500,10 793 414
7,1994,2 129 856,1 146 990,1 340 268,1 122 103,1 399 157,1 389 787,1 222 214,1 082 848,10 833 223
8,1995,2 120 545,1 159 421,1 345 471,1 127 647,1 403 081,1 389 976,1 221 918,1 090 916,10 858 975
9,1996,2 116 512,1 169 827,1 350 539,1 131 399,1 406 949,1 389 668,1 221 145,1 097 809,10 883 848


In [14]:
df_final.to_csv("population_idf-1968-2024.csv",index=False)